# Flux-dependent qubit spectroscopy Experiment

This notebook demonstrates how to perform qubit spectroscopy if the qubit resonance frequnecy depends on a second parameter, e.g. an external flux. 
We perform a 2D sweep of the amplitude of the applied external flux and the frequency of an excitation toen applied to the qubit itself.

## 0. General Imports and Definitions

### 0.1 Python Imports 

In [ ]:
# LabOne Q:
from laboneq.contrib.example_helpers.example_notebook_helper import create_device_setup

# Helpers:
from laboneq.contrib.example_helpers.plotting.plot_helpers import *
from laboneq.simple import *

## 1. Device Setup

In [ ]:
# create device setup
device_setup = create_device_setup(generation=2)
use_emulation = True

In [ ]:
#  signal map for qubits
def map_qubit(qubit_id):
    return {
        "drive": f"/logical_signal_groups/q{qubit_id}/drive_line",
        "flux": f"/logical_signal_groups/q{qubit_id}/flux_line",
        "measure": f"/logical_signal_groups/q{qubit_id}/measure_line",
        "acquire": f"/logical_signal_groups/q{qubit_id}/acquire_line",
    }

## 2. Experiment Definition

In [ ]:
## define pulses

# flux pulse - applied during whole experimental pulse sequence
const_flux = pulse_library.const(uid="const_flux", length=600e-9, amplitude=1.0)
# qubit drive pulse
const_iq_100ns = pulse_library.const(uid="const_iq_100ns", length=100e-9, amplitude=1.0)
# readout drive pulse
readout_pulse = pulse_library.const(uid="readout_pulse", length=400e-9, amplitude=1.0)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=400e-9, amplitude=1.0
)

In [ ]:
# define sweep parameter - sweep over the frequency of a qubit excitation pulse
start = 40e6
stop = 200e6
count = 11

freq_sweep = LinearSweepParameter(
    uid="qubit_frequency", start=start, stop=stop, count=count
)

# Second sweep: Amplitude of the flux pulsed
flux_count = 21

flux_sweep = LinearSweepParameter(uid="flux_qubit", start=0, stop=1, count=flux_count)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

### 2.1 Pulse Sequence

In [ ]:
# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Qubit Flux Spectroscopy",
    signals=[
        ExperimentSignal("flux"),
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
with exp.acquire_loop_rt(
    uid="shots",
    count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC,
    acquisition_type=AcquisitionType.INTEGRATION,
):
    with exp.sweep(uid="sweep", parameter=freq_sweep):
        # inner loop - real-time, sequential averaging in standard integration mode
        # inner loop - adjust flux bias to qubit
        with exp.sweep(uid="flux_sweep", parameter=flux_sweep):
            with exp.section(uid="flux bias"):
                exp.play(signal="flux", pulse=const_flux, amplitude=flux_sweep)
            # qubit excitation pulse - frequency will be swept
            with exp.section(uid="qubit_excitation"):
                # allow for transients to settle
                exp.delay(signal="drive", time=100e-9)
                # play excitation pulse
                exp.play(signal="drive", pulse=const_iq_100ns)
            # readout and data acquisition
            with exp.section(uid="qubit_readout", play_after="qubit_excitation"):
                # play readout pulse
                exp.play(signal="measure", pulse=readout_pulse)
                # signal data acquisition
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to ground state
            with exp.section(uid="relax", play_after="qubit_readout"):
                exp.delay(signal="measure", time=1e-6)

In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=freq_sweep,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_qubit(0))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
my_results = session.run(exp)

In [ ]:
# Plot simulated output signals
plot_simulation(session.compiled_experiment, start_time=0, length=10e-6)

In [ ]:
# plot measurement results
plot_result_3d(my_results, "ac_0")

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
show_pulse_sheet("Qubit Flux Spectroscopy", session.compiled_experiment)

In [ ]:
# set calibration and signal map for qubit 1
exp.set_calibration(exp_calib)
exp.set_signal_map(map_qubit(1))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp)